# FIT5148 - Distributed Databases and Big Data

# Assignment 2 TaskB - Solution Workbook

**Your details:**
- Name: Zhiqing Shu
- Student ID: 28217551

- Name: Yicheng Zhang
- Student ID: 27699641

Let's get started!

**1.Based on the two data sets provided i.e. Fire data-Part 1 and Weather data-Part 1, design a suitable data model to support efficient querying of the two data sets in MongoDB. Justify your data model design.**

We decided to model One-to-Many Relationships between `climate` and `fire` with `Embedded Documents` based on following reasons:

* 'One-to-Some': the number of the fire that broken out in a certain day is impossible to be a large number, so one climate data is associated with just few fire data. In this case, `Documents Reference` is unnecessary.
* In our task, we just need to query rather than update, and embedding provides better performance for read operations since it reduces the work load of I/O of disk.
* We don't need to have a sperate query to get embeded data details.

However, there are some disadvantages for this model:
* The fire cannot be accessed as stand-alone entities without aggregate.
* The statistics for fire data and its field may take more resources.

After considering the pros and cons, we believe that the `Embedded` model is a better choice for our task.

**2.Create a new database in MongoDB. The new database will be based on the document model you have designed in Task B1.**

Based on the above designing, the new database and its document model should be like the following sample:
```json
{'Station': 948701,
  'Date': '2017-04-01',
  'AirTemperature(Celcius)': 14,
  'RelativeHumidity': 45.7,
  'WindSpeed(knots)': 9.2,
  'MaxWindSpeed': 15.9,
  'MAX': '63.9*',
  'MIN': '50.4',
  'Precipitation': '0.00I',
  'fire': [{'Latitude': -36.1948,
    'Longitude': 145.7688,
    'Surface Temperature (kelvin)': 323.8,
    'Datetime': '2017-04-01T04:11:30',
    'Power': 18.7,
    'Confidence': 73,
    'Date': '2017-04-01',
    'Surface Temperature (Celcius)': 50}
```

The fire data store in a list and embedded in the climate data.

**3. Write a python program that will read the data from Fire data-Part1 and Climate data - Part1 and load them to the new database created in Task B2.**

In [1]:
import pymongo
from pymongo import MongoClient
import pandas as pd
from pprint import pprint
import time
import multiprocessing as mp

In [2]:
# connect on the default host and port
client = MongoClient () 
db = client.TaskB_db
# drop collection in case duplicate
db.Combine.drop()
collection = db.Combine 

In [3]:
# read csv as dataframe
climate = pd.read_csv('ClimateData-Part1.csv')
fire = pd.read_csv('FireData-Part1.csv') 

In [4]:
# remove space
climate.columns = climate.columns.str.replace('\s+', '')
climate.replace('\s+', '',regex=True,inplace=True) 

In [5]:
# transfer dataframe to dictionaries
climatelist = climate.to_dict('records')
firelist = fire.to_dict('records') 

In [20]:
climatelist[1]

{'Station': 948700,
 'Date': '2017-01-02',
 'AirTemperature(Celcius)': 15,
 'RelativeHumidity': 50.7,
 'WindSpeed(knots)': 9.2,
 'MaxWindSpeed': 13.0,
 'MAX': '72.0*',
 'MIN': '58.3',
 'Precipitation': '0.02G',
 'fire': [],
 '_id': ObjectId('5b17eb16b3dbfe1420cdbc36')}

In [6]:
# Check data
firelist[1]  

{'Latitude': -35.541,
 'Longitude': 143.311,
 'Surface Temperature (kelvin)': 336.3,
 'Datetime': '2017-12-27T00:02:15',
 'Power': 62.0,
 'Confidence': 82,
 'Date': '2017-12-27',
 'Surface Temperature (Celcius)': 63}

In [7]:
# join two tables
for i,val in enumerate(climatelist):
    climatelist[i]['fire'] = []
    for fire in firelist:
        if climatelist[i]['Date'] == fire['Date']:
            templist = climatelist[i]['fire']
            templist.append(fire)
            climatelist[i]['fire'] = templist 

In [8]:
# check results
climatelist[90:100] 

[{'Station': 948701,
  'Date': '2017-04-01',
  'AirTemperature(Celcius)': 14,
  'RelativeHumidity': 45.7,
  'WindSpeed(knots)': 9.2,
  'MaxWindSpeed': 15.9,
  'MAX': '63.9*',
  'MIN': '50.4',
  'Precipitation': '0.00I',
  'fire': [{'Latitude': -36.1948,
    'Longitude': 145.7688,
    'Surface Temperature (kelvin)': 323.8,
    'Datetime': '2017-04-01T04:11:30',
    'Power': 18.7,
    'Confidence': 73,
    'Date': '2017-04-01',
    'Surface Temperature (Celcius)': 50},
   {'Latitude': -36.2721,
    'Longitude': 146.847,
    'Surface Temperature (kelvin)': 318.9,
    'Datetime': '2017-04-01T04:09:20',
    'Power': 20.2,
    'Confidence': 70,
    'Date': '2017-04-01',
    'Surface Temperature (Celcius)': 45},
   {'Latitude': -36.1017,
    'Longitude': 147.6381,
    'Surface Temperature (kelvin)': 319.2,
    'Datetime': '2017-04-01T04:04:50',
    'Power': 21.6,
    'Confidence': 71,
    'Date': '2017-04-01',
    'Surface Temperature (Celcius)': 46},
   {'Latitude': -36.1256,
    'Longitude'

In [9]:
# drop unnecessary field 
for i in range(len(climatelist)):
    for j in range(len(climatelist[i]['fire'])):
        del climatelist[i]['fire'][j]['Date']  

In [10]:
# Check results
climatelist[90:100] 

[{'Station': 948701,
  'Date': '2017-04-01',
  'AirTemperature(Celcius)': 14,
  'RelativeHumidity': 45.7,
  'WindSpeed(knots)': 9.2,
  'MaxWindSpeed': 15.9,
  'MAX': '63.9*',
  'MIN': '50.4',
  'Precipitation': '0.00I',
  'fire': [{'Latitude': -36.1948,
    'Longitude': 145.7688,
    'Surface Temperature (kelvin)': 323.8,
    'Datetime': '2017-04-01T04:11:30',
    'Power': 18.7,
    'Confidence': 73,
    'Surface Temperature (Celcius)': 50},
   {'Latitude': -36.2721,
    'Longitude': 146.847,
    'Surface Temperature (kelvin)': 318.9,
    'Datetime': '2017-04-01T04:09:20',
    'Power': 20.2,
    'Confidence': 70,
    'Surface Temperature (Celcius)': 45},
   {'Latitude': -36.1017,
    'Longitude': 147.6381,
    'Surface Temperature (kelvin)': 319.2,
    'Datetime': '2017-04-01T04:04:50',
    'Power': 21.6,
    'Confidence': 71,
    'Surface Temperature (Celcius)': 46},
   {'Latitude': -36.1256,
    'Longitude': 145.2444,
    'Surface Temperature (kelvin)': 326.4,
    'Datetime': '2017-0

In [11]:
#insert documents into mongoDB, the new database is named as 'Combine'
db.Combine.insert_many(climatelist) 

**4.Write queries to answer the Tasks A2-A8 on the new database. You need to write the queries as a python program using pymongo library in Jupyter Notebook.**

A2. Find climate data on 15th December 2017.

In [32]:
A0 = db.Combine.find({'AirTemperature(Celcius)':{"$eq":24}},{"fire":1})

In [33]:
for ducoment in A0:
    pprint(ducoment)

{'_id': ObjectId('5b17eb16b3dbfe1420cdbc38'), 'fire': []}
{'_id': ObjectId('5b17eb16b3dbfe1420cdbc39'), 'fire': []}
{'_id': ObjectId('5b17eb16b3dbfe1420cdbc44'), 'fire': []}
{'_id': ObjectId('5b17eb16b3dbfe1420cdbc45'), 'fire': []}
{'_id': ObjectId('5b17eb16b3dbfe1420cdbc6e'), 'fire': []}
{'_id': ObjectId('5b17eb16b3dbfe1420cdbc82'),
 'fire': [{'Confidence': 85,
           'Datetime': '2017-03-19T04:39:50',
           'Latitude': -37.6605,
           'Longitude': 142.1116,
           'Power': 19.2,
           'Surface Temperature (Celcius)': 60,
           'Surface Temperature (kelvin)': 333.3},
          {'Confidence': 58,
           'Datetime': '2017-03-19T04:39:20',
           'Latitude': -37.9164,
           'Longitude': 143.4866,
           'Power': 11.7,
           'Surface Temperature (Celcius)': 54,
           'Surface Temperature (kelvin)': 327.6},
          {'Confidence': 96,
           'Datetime': '2017-03-19T04:39:20',
           'Latitude': -37.6622,
           'Longitude'

In [42]:
A00 = db.Combine.aggregate([{"$unwind":"$fire"},\
                            {"$match":{'AirTemperature(Celcius)':{"$eq":24}}},\
                            {"$project":{"fire":1}}])

In [43]:
for ducoment in A00:
    pprint(ducoment)

{'_id': ObjectId('5b17eb16b3dbfe1420cdbc82'),
 'fire': {'Confidence': 85,
          'Datetime': '2017-03-19T04:39:50',
          'Latitude': -37.6605,
          'Longitude': 142.1116,
          'Power': 19.2,
          'Surface Temperature (Celcius)': 60,
          'Surface Temperature (kelvin)': 333.3}}
{'_id': ObjectId('5b17eb16b3dbfe1420cdbc82'),
 'fire': {'Confidence': 58,
          'Datetime': '2017-03-19T04:39:20',
          'Latitude': -37.9164,
          'Longitude': 143.4866,
          'Power': 11.7,
          'Surface Temperature (Celcius)': 54,
          'Surface Temperature (kelvin)': 327.6}}
{'_id': ObjectId('5b17eb16b3dbfe1420cdbc82'),
 'fire': {'Confidence': 96,
          'Datetime': '2017-03-19T04:39:20',
          'Latitude': -37.6622,
          'Longitude': 142.1001,
          'Power': 55.4,
          'Surface Temperature (Celcius)': 79,
          'Surface Temperature (kelvin)': 352.2}}
{'_id': ObjectId('5b17eb16b3dbfe1420cdbc82'),
 'fire': {'Confidence': 63,
        

In [12]:
# db.climate.find({ 'Date': '2017-12-15'} , {'_id': 0}).pretty()

In [18]:
A2 = db.Combine.find({"Date":"2017-12-15"},{'_id':0,'fire':0})

In [19]:
for ducoment in A2:
    pprint(ducoment)

{'AirTemperature(Celcius)': 18,
 'Date': '2017-12-15',
 'MAX': '74.5*',
 'MIN': '53.1',
 'MaxWindSpeed': 14.0,
 'Precipitation': '0.00I',
 'RelativeHumidity': 52.0,
 'Station': 948702,
 'WindSpeed(knots)': 7.1}


A3. Find the latitude, longitude and confidence when the surface temperature (°C) was between 65 °C and 100 °C.

In [ ]:
# db.fire.find({'Surface Temperature (Celcius)':{$gte:65, $lte:100}},
            # {'Latitude':1,'Longitude':1,'Confidence':1,
            # 'Surface Temperature (Celcius)': 1,'_id':0}).pretty()

In [14]:
#Unwind fire to ensure retrieve fire data as stand-alone record
#Match surface temperature between 65 and 100
#Project specified field
A3 = db.Combine.aggregate([{"$unwind": "$fire"},\
                           {"$match": {"fire.Surface Temperature (Celcius)": { "$gte":65,"$lte":100}}},\
                           {"$project":{'fire.Latitude':1,'fire.Longitude': 1,'fire.Confidence':1,\
                            'fire.Surface Temperature (Celcius)':1,'_id':0}}])

In [15]:
for ducoment in A3:
    pprint(ducoment)

{'fire': {'Confidence': 94,
          'Latitude': -37.2284,
          'Longitude': 147.9187,
          'Surface Temperature (Celcius)': 73}}
{'fire': {'Confidence': 97,
          'Latitude': -37.6572,
          'Longitude': 142.0703,
          'Surface Temperature (Celcius)': 80}}
{'fire': {'Confidence': 84,
          'Latitude': -37.0193,
          'Longitude': 148.1459,
          'Surface Temperature (Celcius)': 71}}
{'fire': {'Confidence': 100,
          'Latitude': -37.4229,
          'Longitude': 147.02700000000002,
          'Surface Temperature (Celcius)': 99}}
{'fire': {'Confidence': 80,
          'Latitude': -37.0055,
          'Longitude': 148.1582,
          'Surface Temperature (Celcius)': 68}}
{'fire': {'Confidence': 85,
          'Latitude': -37.4128,
          'Longitude': 147.0242,
          'Surface Temperature (Celcius)': 98}}
{'fire': {'Confidence': 90,
          'Latitude': -34.357,
          'Longitude': 141.5361,
          'Surface Temperature (Celcius)': 67}}
{'f

{'fire': {'Confidence': 94,
          'Latitude': -36.2933,
          'Longitude': 141.36,
          'Surface Temperature (Celcius)': 75}}
{'fire': {'Confidence': 100,
          'Latitude': -37.4591,
          'Longitude': 144.9492,
          'Surface Temperature (Celcius)': 89}}
{'fire': {'Confidence': 91,
          'Latitude': -36.3487,
          'Longitude': 145.5315,
          'Surface Temperature (Celcius)': 68}}
{'fire': {'Confidence': 90,
          'Latitude': -36.5399,
          'Longitude': 144.678,
          'Surface Temperature (Celcius)': 66}}
{'fire': {'Confidence': 97,
          'Latitude': -36.8158,
          'Longitude': 142.6229,
          'Surface Temperature (Celcius)': 80}}
{'fire': {'Confidence': 100,
          'Latitude': -36.4518,
          'Longitude': 144.7752,
          'Surface Temperature (Celcius)': 97}}
{'fire': {'Confidence': 90,
          'Latitude': -36.7903,
          'Longitude': 141.618,
          'Surface Temperature (Celcius)': 67}}
{'fire': {'Conf

          'Latitude': -36.5854,
          'Longitude': 143.0462,
          'Surface Temperature (Celcius)': 94}}
{'fire': {'Confidence': 95,
          'Latitude': -36.3014,
          'Longitude': 143.203,
          'Surface Temperature (Celcius)': 76}}
{'fire': {'Confidence': 100,
          'Latitude': -38.1917,
          'Longitude': 147.0745,
          'Surface Temperature (Celcius)': 94}}
{'fire': {'Confidence': 95,
          'Latitude': -38.1987,
          'Longitude': 147.0666,
          'Surface Temperature (Celcius)': 76}}
{'fire': {'Confidence': 100,
          'Latitude': -36.5454,
          'Longitude': 144.7402,
          'Surface Temperature (Celcius)': 96}}
{'fire': {'Confidence': 92,
          'Latitude': -36.7593,
          'Longitude': 142.895,
          'Surface Temperature (Celcius)': 69}}
{'fire': {'Confidence': 90,
          'Latitude': -36.3435,
          'Longitude': 141.5513,
          'Surface Temperature (Celcius)': 67}}
{'fire': {'Confidence': 93,
          'La

A4.Find surface temperature (°C), air temperature (°C), relative humidity and maximum wind speed on 15th and 16th of December 2017.

In [ ]:
# db.climate.aggregate([{
#     $match: { "Date": { $in: [ "2017-12-15", "2017-12-16" ] } }
#     },
#     {
#         $lookup:
#         {
#             from: "fire",
#             localField : "Date",
#             foreignField : "Date",
#             as : "FireData"
#         }
#     },
#     {
#         $unwind: "$FireData"
#     },
#     {
#         $replaceRoot: { newRoot: { $mergeObjects: [ "$FireData", "$$ROOT" ] } }
#     },
#     {
#         $project: 
#         { 
#             "_id": 0,
#             "Surface Temperature (Celcius)" : 1, 
#             "Air Temperature(Celcius)": 1,
#             "Relative Humidity": 1,
#             "Date": 1,
#             "Max Wind Speed": 1
#         }
#     }
# ]).pretty()

In [16]:
#Match Date in 2017-12-15 and 2017-12-16
#Project specified field
A4 = db.Combine.find({'Date': {"$in":["2017-12-15", "2017-12-16"]}},\
                     {'_id': 0,'Date':1,'AirTemperature(Celcius)':1,'MaxWindSpeed':1,\
                     'RelativeHumidity':1,'fire.Surface Temperature (Celcius)':1})

In [17]:
for ducoment in A4:
    pprint(ducoment)

{'AirTemperature(Celcius)': 18,
 'Date': '2017-12-15',
 'MaxWindSpeed': 14.0,
 'RelativeHumidity': 52.0,
 'fire': [{'Surface Temperature (Celcius)': 42},
          {'Surface Temperature (Celcius)': 36},
          {'Surface Temperature (Celcius)': 38},
          {'Surface Temperature (Celcius)': 40}]}
{'AirTemperature(Celcius)': 18,
 'Date': '2017-12-16',
 'MaxWindSpeed': 13.0,
 'RelativeHumidity': 53.7,
 'fire': [{'Surface Temperature (Celcius)': 43},
          {'Surface Temperature (Celcius)': 33},
          {'Surface Temperature (Celcius)': 54},
          {'Surface Temperature (Celcius)': 73},
          {'Surface Temperature (Celcius)': 55},
          {'Surface Temperature (Celcius)': 75},
          {'Surface Temperature (Celcius)': 55},
          {'Surface Temperature (Celcius)': 66},
          {'Surface Temperature (Celcius)': 56},
          {'Surface Temperature (Celcius)': 60},
          {'Surface Temperature (Celcius)': 73},
          {'Surface Temperature (Celcius)': 48},
     

A5.Find datetime, air temperature (°C), surface temperature (°C) and confidence when the confidence is between 80 and 100.

In [ ]:
# db.fire.aggregate([
#     {
#         $match: {'Confidence':{$gte:80, $lte:100}}
#     },
#     {
#         $lookup:
#         {
#             from: "climate",
#             localField: "Date",
#             foreignField: "Date",
#             as:"climate"
#         }
#     },
#     {
#         $unwind: "$climate"
#     },
#     {
#         $replaceRoot: {newRoot: {$mergeObjects: ["$climate","$$ROOT"]}}
#     },
#     {
#         $project: 
#         { 
#             "_id": 0,
#             "Datetime" : 1, 
#             "Air Temperature(Celcius)": 1,
#             "Surface Temperature (Celcius)": 1,
#             "Confidence": 1
#         }
#     }
# ]).pretty()

In [18]:
#Unwind fire to ensure retrieve fire data as stand-alone record
#Match surface confidence between 80 and 100
#Project specified field
A5 = db.Combine.aggregate([{"$unwind":"$fire"},\
                            {"$match":{"fire.Confidence":{ "$gte":80,"$lte":100}}},\
                            {"$project":{'fire.Datetime':1,'AirTemperature(Celcius)':1,\
                            'fire.Confidence':1,'fire.Surface Temperature (Celcius)':1,'_id':0}}])

In [19]:
for document in A5: 
    pprint(document)

{'AirTemperature(Celcius)': 20,
 'fire': {'Confidence': 87,
          'Datetime': '2017-03-06T05:06:30',
          'Surface Temperature (Celcius)': 62}}
{'AirTemperature(Celcius)': 20,
 'fire': {'Confidence': 85,
          'Datetime': '2017-03-06T05:06:20',
          'Surface Temperature (Celcius)': 59}}
{'AirTemperature(Celcius)': 19,
 'fire': {'Confidence': 88,
          'Datetime': '2017-03-07T04:16:10',
          'Surface Temperature (Celcius)': 64}}
{'AirTemperature(Celcius)': 23,
 'fire': {'Confidence': 86,
          'Datetime': '2017-03-09T13:23:40',
          'Surface Temperature (Celcius)': 41}}
{'AirTemperature(Celcius)': 19,
 'fire': {'Confidence': 100,
          'Datetime': '2017-03-10T04:48:40',
          'Surface Temperature (Celcius)': 105}}
{'AirTemperature(Celcius)': 19,
 'fire': {'Confidence': 100,
          'Datetime': '2017-03-10T04:46:20',
          'Surface Temperature (Celcius)': 109}}
{'AirTemperature(Celcius)': 19,
 'fire': {'Confidence': 94,
          'Datetim

 'fire': {'Confidence': 82,
          'Datetime': '2017-04-04T04:41:10',
          'Surface Temperature (Celcius)': 62}}
{'AirTemperature(Celcius)': 16,
 'fire': {'Confidence': 88,
          'Datetime': '2017-04-04T04:41:00',
          'Surface Temperature (Celcius)': 64}}
{'AirTemperature(Celcius)': 16,
 'fire': {'Confidence': 86,
          'Datetime': '2017-04-04T04:41:00',
          'Surface Temperature (Celcius)': 61}}
{'AirTemperature(Celcius)': 16,
 'fire': {'Confidence': 100,
          'Datetime': '2017-04-04T04:40:00',
          'Surface Temperature (Celcius)': 120}}
{'AirTemperature(Celcius)': 16,
 'fire': {'Confidence': 90,
          'Datetime': '2017-04-04T04:39:50',
          'Surface Temperature (Celcius)': 67}}
{'AirTemperature(Celcius)': 16,
 'fire': {'Confidence': 85,
          'Datetime': '2017-04-04T04:39:50',
          'Surface Temperature (Celcius)': 66}}
{'AirTemperature(Celcius)': 16,
 'fire': {'Confidence': 90,
          'Datetime': '2017-04-04T04:39:40',
       

          'Surface Temperature (Celcius)': 59}}
{'AirTemperature(Celcius)': 19,
 'fire': {'Confidence': 98,
          'Datetime': '2017-04-08T04:08:50',
          'Surface Temperature (Celcius)': 83}}
{'AirTemperature(Celcius)': 19,
 'fire': {'Confidence': 90,
          'Datetime': '2017-04-08T04:08:30',
          'Surface Temperature (Celcius)': 66}}
{'AirTemperature(Celcius)': 19,
 'fire': {'Confidence': 80,
          'Datetime': '2017-04-08T04:08:30',
          'Surface Temperature (Celcius)': 54}}
{'AirTemperature(Celcius)': 19,
 'fire': {'Confidence': 80,
          'Datetime': '2017-04-08T04:08:30',
          'Surface Temperature (Celcius)': 53}}
{'AirTemperature(Celcius)': 19,
 'fire': {'Confidence': 80,
          'Datetime': '2017-04-08T04:08:30',
          'Surface Temperature (Celcius)': 58}}
{'AirTemperature(Celcius)': 14,
 'fire': {'Confidence': 81,
          'Datetime': '2017-04-11T04:39:20',
          'Surface Temperature (Celcius)': 54}}
{'AirTemperature(Celcius)': 14,
 '

          'Datetime': '2017-04-13T04:26:30',
          'Surface Temperature (Celcius)': 58}}
{'AirTemperature(Celcius)': 16,
 'fire': {'Confidence': 84,
          'Datetime': '2017-04-13T04:26:30',
          'Surface Temperature (Celcius)': 58}}
{'AirTemperature(Celcius)': 16,
 'fire': {'Confidence': 86,
          'Datetime': '2017-04-13T04:26:30',
          'Surface Temperature (Celcius)': 61}}
{'AirTemperature(Celcius)': 16,
 'fire': {'Confidence': 100,
          'Datetime': '2017-04-13T04:26:30',
          'Surface Temperature (Celcius)': 92}}
{'AirTemperature(Celcius)': 16,
 'fire': {'Confidence': 83,
          'Datetime': '2017-04-13T04:26:30',
          'Surface Temperature (Celcius)': 56}}
{'AirTemperature(Celcius)': 16,
 'fire': {'Confidence': 100,
          'Datetime': '2017-04-13T04:26:30',
          'Surface Temperature (Celcius)': 111}}
{'AirTemperature(Celcius)': 16,
 'fire': {'Confidence': 100,
          'Datetime': '2017-04-13T04:26:30',
          'Surface Temperature (C

 'fire': {'Confidence': 93,
          'Datetime': '2017-04-18T04:44:50',
          'Surface Temperature (Celcius)': 73}}
{'AirTemperature(Celcius)': 15,
 'fire': {'Confidence': 93,
          'Datetime': '2017-04-18T04:44:50',
          'Surface Temperature (Celcius)': 72}}
{'AirTemperature(Celcius)': 15,
 'fire': {'Confidence': 80,
          'Datetime': '2017-04-18T04:44:50',
          'Surface Temperature (Celcius)': 53}}
{'AirTemperature(Celcius)': 15,
 'fire': {'Confidence': 94,
          'Datetime': '2017-04-18T04:44:50',
          'Surface Temperature (Celcius)': 75}}
{'AirTemperature(Celcius)': 15,
 'fire': {'Confidence': 84,
          'Datetime': '2017-04-18T04:44:50',
          'Surface Temperature (Celcius)': 58}}
{'AirTemperature(Celcius)': 15,
 'fire': {'Confidence': 89,
          'Datetime': '2017-04-18T04:44:50',
          'Surface Temperature (Celcius)': 65}}
{'AirTemperature(Celcius)': 15,
 'fire': {'Confidence': 91,
          'Datetime': '2017-04-18T00:30:50',
         

{'AirTemperature(Celcius)': 12,
 'fire': {'Confidence': 83,
          'Datetime': '2017-05-08T04:20:20',
          'Surface Temperature (Celcius)': 56}}
{'AirTemperature(Celcius)': 12,
 'fire': {'Confidence': 86,
          'Datetime': '2017-05-08T04:20:10',
          'Surface Temperature (Celcius)': 60}}
{'AirTemperature(Celcius)': 12,
 'fire': {'Confidence': 91,
          'Datetime': '2017-05-08T04:20:10',
          'Surface Temperature (Celcius)': 68}}
{'AirTemperature(Celcius)': 12,
 'fire': {'Confidence': 100,
          'Datetime': '2017-05-08T04:20:10',
          'Surface Temperature (Celcius)': 101}}
{'AirTemperature(Celcius)': 12,
 'fire': {'Confidence': 100,
          'Datetime': '2017-05-08T04:20:10',
          'Surface Temperature (Celcius)': 115}}
{'AirTemperature(Celcius)': 12,
 'fire': {'Confidence': 97,
          'Datetime': '2017-05-08T04:20:10',
          'Surface Temperature (Celcius)': 81}}
{'AirTemperature(Celcius)': 12,
 'fire': {'Confidence': 81,
          'Datetim

          'Surface Temperature (Celcius)': 112}}
{'AirTemperature(Celcius)': 14,
 'fire': {'Confidence': 88,
          'Datetime': '2017-09-20T04:28:45',
          'Surface Temperature (Celcius)': 60}}
{'AirTemperature(Celcius)': 17,
 'fire': {'Confidence': 86,
          'Datetime': '2017-09-23T23:50:44',
          'Surface Temperature (Celcius)': 56}}
{'AirTemperature(Celcius)': 17,
 'fire': {'Confidence': 100,
          'Datetime': '2017-09-23T12:47:12',
          'Surface Temperature (Celcius)': 56}}
{'AirTemperature(Celcius)': 17,
 'fire': {'Confidence': 100,
          'Datetime': '2017-09-23T12:47:12',
          'Surface Temperature (Celcius)': 65}}
{'AirTemperature(Celcius)': 17,
 'fire': {'Confidence': 100,
          'Datetime': '2017-09-23T12:47:12',
          'Surface Temperature (Celcius)': 48}}
{'AirTemperature(Celcius)': 17,
 'fire': {'Confidence': 100,
          'Datetime': '2017-09-23T12:47:12',
          'Surface Temperature (Celcius)': 52}}
{'AirTemperature(Celcius)': 1

A6. Find top 10 records with highest surface temperature (°C).

In [ ]:
# db.fire.find({},{"_id":0}).sort({"Surface Temperature (Celcius)":-1}).limit(10).pretty();

In [20]:
#Unwind fire to ensure retrieve fire data as stand-alone record
#Sort by surface temperature in descending order(max to min)
#Project specified field
A6 = db.Combine.aggregate([{"$unwind": "$fire"},\
                            {"$sort": {"fire.Surface Temperature (Celcius)":-1}},\
                            {"$limit":10},\
                            {"$project":{'_id':0}}])

In [21]:
for document in A6: 
    pprint(document)

{'AirTemperature(Celcius)': 15,
 'Date': '2017-04-18',
 'MAX': '69.8*',
 'MIN': '46.8',
 'MaxWindSpeed': 9.9,
 'Precipitation': '0.00I',
 'RelativeHumidity': 56.1,
 'Station': 948701,
 'WindSpeed(knots)': 5.1,
 'fire': {'Confidence': 100,
          'Datetime': '2017-04-18T04:52:00',
          'Latitude': -38.1665,
          'Longitude': 143.062,
          'Power': 239.8,
          'Surface Temperature (Celcius)': 124,
          'Surface Temperature (kelvin)': 397.5}}
{'AirTemperature(Celcius)': 16,
 'Date': '2017-04-04',
 'MAX': '73.4*',
 'MIN': '46.8',
 'MaxWindSpeed': 12.0,
 'Precipitation': '0.00I',
 'RelativeHumidity': 47.5,
 'Station': 948701,
 'WindSpeed(knots)': 5.4,
 'fire': {'Confidence': 100,
          'Datetime': '2017-04-04T04:32:50',
          'Latitude': -36.343,
          'Longitude': 142.1986,
          'Power': 233.4,
          'Surface Temperature (Celcius)': 123,
          'Surface Temperature (kelvin)': 396.3}}
{'AirTemperature(Celcius)': 14,
 'Date': '2017-05-01',


A7. Find the number of fire in each day. You are required to only display total number of fire
and the date in the output.

In [ ]:
# db.fire.aggregate(
# [
#     {
#         $group : 
#         {
#             _id : "$Date",
#             "Firecount" : {$sum : 1}
#         }
#     },
#     {
#         $sort :
#         {
#             '_id': 1
#         }
#     },
#     { 
#         $project: 
#         {  
#             _id: 0,
#             "Date": "$_id",
#             Firecount: 1
#         }
#     }
# ])

In [22]:
#Unwind fire to ensure retrieve fire data as stand-alone record
#Count only date has fire
#group by date and caculate sum
#sort by date
#Project specified field
A7 = db.Combine.aggregate([{"$unwind": "$fire"},\
                            {"$match": {"fire" : {"$ne": []}}},\
                            {"$group":{'_id':'$Date','Firecount':{'$sum':1}}},\
                            {"$sort":{'_id':1}},\
                            {"$project":{'_id':0,"Date":'$_id','Firecount':1}}])

In [23]:
for document in A7: 
    pprint(document)

{'Date': '2017-03-06', 'Firecount': 2}
{'Date': '2017-03-07', 'Firecount': 1}
{'Date': '2017-03-08', 'Firecount': 2}
{'Date': '2017-03-09', 'Firecount': 3}
{'Date': '2017-03-10', 'Firecount': 8}
{'Date': '2017-03-12', 'Firecount': 5}
{'Date': '2017-03-13', 'Firecount': 2}
{'Date': '2017-03-14', 'Firecount': 10}
{'Date': '2017-03-15', 'Firecount': 7}
{'Date': '2017-03-17', 'Firecount': 6}
{'Date': '2017-03-18', 'Firecount': 3}
{'Date': '2017-03-19', 'Firecount': 21}
{'Date': '2017-03-24', 'Firecount': 2}
{'Date': '2017-03-25', 'Firecount': 13}
{'Date': '2017-03-26', 'Firecount': 17}
{'Date': '2017-03-28', 'Firecount': 54}
{'Date': '2017-03-29', 'Firecount': 1}
{'Date': '2017-03-31', 'Firecount': 22}
{'Date': '2017-04-01', 'Firecount': 7}
{'Date': '2017-04-02', 'Firecount': 5}
{'Date': '2017-04-03', 'Firecount': 72}
{'Date': '2017-04-04', 'Firecount': 89}
{'Date': '2017-04-05', 'Firecount': 49}
{'Date': '2017-04-06', 'Firecount': 118}
{'Date': '2017-04-07', 'Firecount': 39}
{'Date': '201

A8. Find the average surface temperature (°C) for each day. You are required to only display
average surface temperature (°C) and the date in the output.

In [ ]:
# db.fire.aggregate(
# [
#     {
#         $group : 
#         {
#             _id : "$Date",
#             "Avg temperature" : {$avg : "$Surface Temperature (Celcius)"}
#         }
#     },
#     {
#         $sort :
#         {
#             '_id': 1
#         }
#     },
#     { 
#         $project: 
#         {  
#             _id: 0,
#             "Date" : "$_id",
#             "Avg temperature" : 1
#         }
#     }
# ])


In [24]:
#Unwind fire to ensure retrieve fire data as stand-alone record
#Count only date has fire
#group by date and caculate Average
#sort by date
#Project specified field
A8 = db.Combine.aggregate([{"$unwind": "$fire"},\
                            {"$match": {"fire" : {"$ne": []}}},\
                            {"$group":{'_id':'$Date','Avg Temperature':{'$avg':"$fire.Surface Temperature (Celcius)"}}},\
                            {"$sort":{'_id':1}},\
                            {"$project":{'_id':0,"Date":'$_id','Avg Temperature':1}}])

In [25]:
for document in A8: 
    pprint(document)

{'Avg Temperature': 60.5, 'Date': '2017-03-06'}
{'Avg Temperature': 64.0, 'Date': '2017-03-07'}
{'Avg Temperature': 51.5, 'Date': '2017-03-08'}
{'Avg Temperature': 46.666666666666664, 'Date': '2017-03-09'}
{'Avg Temperature': 69.375, 'Date': '2017-03-10'}
{'Avg Temperature': 88.2, 'Date': '2017-03-12'}
{'Avg Temperature': 38.5, 'Date': '2017-03-13'}
{'Avg Temperature': 65.6, 'Date': '2017-03-14'}
{'Avg Temperature': 46.0, 'Date': '2017-03-15'}
{'Avg Temperature': 59.5, 'Date': '2017-03-17'}
{'Avg Temperature': 79.33333333333333, 'Date': '2017-03-18'}
{'Avg Temperature': 65.57142857142857, 'Date': '2017-03-19'}
{'Avg Temperature': 49.0, 'Date': '2017-03-24'}
{'Avg Temperature': 66.0, 'Date': '2017-03-25'}
{'Avg Temperature': 56.88235294117647, 'Date': '2017-03-26'}
{'Avg Temperature': 60.925925925925924, 'Date': '2017-03-28'}
{'Avg Temperature': 51.0, 'Date': '2017-03-29'}
{'Avg Temperature': 48.72727272727273, 'Date': '2017-03-31'}
{'Avg Temperature': 46.714285714285715, 'Date': '2017-

**Please choose ONE of the Task A2-A8 and implement it in parallel.
A7. Find the number of fire in each day. You are required to only display total number of fire and the date in the output.**

In [26]:
#convert entire 'Combine' database to list
result = list(db.Combine.find())

In [27]:
result[90:100]

[{'_id': ObjectId('5b019c55b3dbfe3de262dcbf'),
  'Station': 948701,
  'Date': '2017-04-01',
  'AirTemperature(Celcius)': 14,
  'RelativeHumidity': 45.7,
  'WindSpeed(knots)': 9.2,
  'MaxWindSpeed': 15.9,
  'MAX': '63.9*',
  'MIN': '50.4',
  'Precipitation': '0.00I',
  'fire': [{'Latitude': -36.1948,
    'Longitude': 145.7688,
    'Surface Temperature (kelvin)': 323.8,
    'Datetime': '2017-04-01T04:11:30',
    'Power': 18.7,
    'Confidence': 73,
    'Surface Temperature (Celcius)': 50},
   {'Latitude': -36.2721,
    'Longitude': 146.847,
    'Surface Temperature (kelvin)': 318.9,
    'Datetime': '2017-04-01T04:09:20',
    'Power': 20.2,
    'Confidence': 70,
    'Surface Temperature (Celcius)': 45},
   {'Latitude': -36.1017,
    'Longitude': 147.6381,
    'Surface Temperature (kelvin)': 319.2,
    'Datetime': '2017-04-01T04:04:50',
    'Power': 21.6,
    'Confidence': 71,
    'Surface Temperature (Celcius)': 46},
   {'Latitude': -36.1256,
    'Longitude': 145.2444,
    'Surface Temper

**A7. Find the number of fire in each day. You are required to only display total number of fire and the date in the output.**

In [28]:
#perform round-robin partitioning
def rr_partition(data, n): 
    result = []
    
    # divide data in to n partitions
    for i in range(n): 
        result.append([])
    # calculate the number of the elements to be allocated to each bin
    n_bin = len(data)/n
    
    # for each bin, perform the following
    for index, element in enumerate(data): 
        # calculate the index of the bin that the current data point will be assigned
        index_bin = (int) (index % n)
        result[index_bin].append(element)
    return result

In [29]:
#Local caculation for each partitioning
def calculate(dataset):
    
    result = {}
    
    for r in dataset:
        key =  r['Date']
        val = len(r['fire'])
        if val != 0:
            if key not in result:
                result[key] = 0
            result[key] += val
    
    return result

In [30]:
def parallel_merge_all_groupby(dataset, n_processor):
    
    result = []
    
    subsets = rr_partition(dataset, n_processor)
    
    pool = mp.Pool(processes = n_processor)

    for s in subsets:
        result.append(pool.apply(calculate,[s]))
        
    return result

In [31]:
parallel_merge_all_groupby(result, 3)

[{'2017-03-08': 2,
  '2017-03-14': 10,
  '2017-03-17': 6,
  '2017-03-26': 17,
  '2017-03-29': 1,
  '2017-04-01': 7,
  '2017-04-04': 89,
  '2017-04-07': 39,
  '2017-04-13': 357,
  '2017-04-16': 18,
  '2017-04-19': 50,
  '2017-04-22': 2,
  '2017-04-25': 3,
  '2017-05-01': 20,
  '2017-05-04': 135,
  '2017-05-07': 3,
  '2017-05-10': 114,
  '2017-05-13': 54,
  '2017-05-16': 3,
  '2017-05-22': 33,
  '2017-06-03': 2,
  '2017-06-09': 3,
  '2017-06-18': 2,
  '2017-06-30': 6,
  '2017-07-06': 3,
  '2017-08-02': 2,
  '2017-08-05': 1,
  '2017-08-14': 5,
  '2017-09-10': 4,
  '2017-09-22': 1,
  '2017-10-01': 8,
  '2017-10-04': 5,
  '2017-10-07': 1,
  '2017-10-10': 3,
  '2017-10-16': 1,
  '2017-10-28': 1,
  '2017-11-09': 10,
  '2017-11-12': 5,
  '2017-11-21': 1,
  '2017-11-30': 31,
  '2017-12-09': 4,
  '2017-12-12': 1,
  '2017-12-15': 4,
  '2017-12-21': 1,
  '2017-12-24': 1,
  '2017-12-27': 4},
 {'2017-03-06': 2,
  '2017-03-09': 3,
  '2017-03-12': 5,
  '2017-03-15': 7,
  '2017-03-18': 3,
  '2017-03-24

In [32]:
#Drop collection we created.
db.Combine.drop()